In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Leitura dos Dados
file1 = "/content/PicMoney-Base_de_Transa__es_-_Cupons_Capturados-100000 linhas (1).csv"
file2 = "/content/PicMoney-Base_Cadastral_de_Players-10_000 linhas (1).csv"
file3 = "/content/PicMoney-Base_Simulada_-_Pedestres_Av__Paulista-100000 linhas (1).csv"

df_trans = pd.read_csv(file1, sep=";")
df_cad = pd.read_csv(file2, sep=";")
df_sim = pd.read_csv(file3, sep=";")

# Junção dos Dados
df = df_trans.merge(df_cad, on="celular", how="left")
df = df.merge(df_sim[["celular", "possui_app_picmoney", "idade", "sexo"]],
              on="celular", how="left", suffixes=("", "_sim"))

# Limpeza e Preenchimento dos Dados
df["valor_cupom"] = pd.to_numeric(df["valor_cupom"], errors="coerce")
df["idade"] = pd.to_numeric(df["idade"], errors="coerce")

df["idade"] = df["idade"].fillna(df["idade"].median())
df["valor_cupom"] = df["valor_cupom"].fillna(df["valor_cupom"].median())
df["sexo"] = df["sexo"].fillna("Desconhecido")
df["possui_app_picmoney"] = df["possui_app_picmoney"].fillna("Não informado")

# Convertendo variáveis categóricas em dummy (0/1)
df = pd.get_dummies(df, columns=["sexo", "possui_app_picmoney"], drop_first=True)

# Ajuste do Modelo de Regressão
X = df[["idade", "sexo_Masculino", "possui_app_picmoney_Sim"]].astype(float)
y = df["valor_cupom"].astype(float)

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

# Resultados
print("\n=== Resumo do Modelo ===")
print(model.summary())

print("\n=== Intervalos de Confiança (95%) ===")
print(model.conf_int(alpha=0.05))


# Relatório Resumido

relatorio = """
Relatório – Análise de Regressão Linear (Versão Resumida)

Foi conduzido um modelo de regressão linear para avaliar a relação entre o valor dos cupons capturados (valor_cupom)
e algumas características dos usuários: idade, sexo e possui_app_picmoney.

O modelo foi ajustado usando o método dos Mínimos Quadrados Ordinários (OLS), considerando 100.011 observações.

Resultados Principais:
- R² = 0,000 → o modelo explica praticamente 0% da variação do valor dos cupons.
- Prob(F-statistic) = 0,816 → o modelo global não é estatisticamente significativo.

Coeficientes:
- Constante: 550,36
- Idade: -0,0055
- Sexo_Masculino: 1,20
- Possui_app_picmoney_Sim: 15,34
Todos os intervalos de confiança (95%) incluem zero, indicando ausência de significância estatística.

Interpretação e Conclusão:
Os resultados mostram que idade, sexo e possuir o aplicativo PicMoney não têm impacto relevante sobre o valor dos cupons.
O baixo poder explicativo do modelo sugere que outras variáveis podem ser mais importantes.
Mesmo sem significância estatística, o exercício permitiu praticar a construção, ajuste e interpretação de um modelo de regressão linear.
Isso cumpre os objetivos da atividade e fornece uma base para discussões futuras sobre quais variáveis podem realmente explicar o valor dos cupons.
"""
print(relatorio)
